**加载训练好的词向量模型**

In [2]:
from gensim.models import word2vec
model = word2vec.Word2Vec.load('model_191115_1')

In [3]:
# test
model.wv.most_similar(positive=['女人', '国王'], negative=['男人'])

[('王后', 0.7191973328590393),
 ('王储', 0.6588001847267151),
 ('七世', 0.6451687216758728),
 ('君主', 0.6411615014076233),
 ('王室', 0.6265252232551575),
 ('摄政王', 0.6223193407058716),
 ('六世', 0.6161067485809326),
 ('三世', 0.613813042640686),
 ('王太子', 0.6124220490455627),
 ('一世', 0.6119545102119446)]

In [31]:
model.wv['结束'].shape

(250,)

**词频**

In [14]:
total = 0
for k in model.wv.vocab.keys():
    total += model.wv.vocab[k].count

In [15]:
total

223974885

In [17]:
def get_fre(w):
    
    return model.wv.vocab[w].count / total

**smooth inverse frequency (SIF)**

> **Input:** Word embeddings $\{v_w:w \in V\}$, a set of sentences $S$, parameter $a$ and estimated probabilities $\{p(w):w \in V\}$ of the words. <br>
**Output:** Sentence embeddings $\{v_s:s \in S\}$
>> **Step 1**<br>
**for all** sentence $s$ in $S$ **do**<br>
$v_s \leftarrow\frac{1}{|s|}\sum_{w \in s} \frac{a}{a+p(w)}v_w$<br>
**end for**<br>
>
>>**Step 2**<br>
Form a matrix $X$ whose columns are $\{v_s:s \in S\}$, and let $u$ be its first singular vector<br>
**for all** sentence $s$ in $S$ **do**<br>
$v_s \leftarrow v_s - uu^Tv_s$<br>
**end for**

In [21]:
import numpy as np

In [175]:
# Step 1
def sif_s1(s:list, a=1e-3) -> np.array:
    v = np.zeros(model.wv['算法'].shape)
    count = 0
    for w in s:
        if w not in model.wv: continue
        we_w = a / (a + get_fre(w)) * model.wv[w]
        v += we_w
        count += 1
    return v / count

In [43]:
from sklearn.decomposition import PCA

In [179]:
# Step 2
def sif_s2(s:np.array) -> np.array:
    pca = PCA(n_components=1)
    pca.fit(s)
    pc = pca.components_
    return s - s.dot(pc.T) * pc

**测试SIF**

*数据选择*

In [106]:
import pandas as pd

In [109]:
news = pd.read_csv('sqlResult_1558435.csv', encoding='gb18030')

In [117]:
title = news.iloc[31,:]['title']
content = news.iloc[31,:]['content']

In [123]:
title

'安倍再陷办学丑闻 日内阁府与文科省调查大相径庭'

In [124]:
content

'据中国之声《新闻纵横》报道，针对近日日本媒体披露政府或为首相安倍晋三的好友办学“开绿灯”一事，日本文部科学大臣松野博一日前证实，在文部科学省内部找到已被曝光的大部分文件内容，并针对此前的调查没能得出结论向公众致歉。\r\n据悉，这些内部文件内容涉及由安倍好友担任理事长的加计学园计划利用“国家战略特区”政策设立兽医学院一事。文件中说，加速推进设立兽医学院是“来自官邸最高层的指示”。\r\n不过，日本内阁府16号也公布了有关该事件的内部调查报告。令人匪夷所思的是，两份出自日本政府不同部门的就同一事件进行的调查报告，竟然得出了相互矛盾的结果。这究竟是怎么一回事？这些内部文件和报告中暗藏着那些秘密？\r\n事件要从日本《朝日新闻》今年5月17日披露的内部文件说起，当时被《朝日新闻》披露的部分被怀疑是文部科学省的内部文件中，内容涉及由日本首相安倍晋三的好友担任理事长的加计学园，计划利用“国家战略特区”政策设立兽医学院一事。文件中说，加速推进设立兽医学院是“来自官邸最高层的指示”。\r\n据媒体报道，日本政府已有50多年没有批准新设立兽医学院。今年1月，加计学园获批利用“国家战略特区”政策在爱媛县今治市新设兽医学院。3月，今治市议会通过相关议案，决定为加计学园无偿提供建设用地，并将提供最高达96亿日元的建设补助。对此，前文部科学省事务次官前川喜平5月25日晚召开记者会证实这批文件确实存在。他认为文件中所提到的“官邸最高层”指的应是首相本人或官房长官，并批评政府本应公平、公正的行政环境已经遭到破坏。\r\n关于这份内部文件的真伪，日本文部科学省在第一时间宣称相关文件并不存在，但在已经辞职的元政务官和多位在职职员的证言下，本月15日又改口承认，确实有这样一份文件，并且承认在批准新设兽医学科过程中，受到了来自首相官邸的压力。\r\n对此，日本内阁府迅速展开自查，16日内阁府发布调查资料，称未能发现相关文件，负责主导相关调查的地方创生大臣山本幸三甚至在国会暗示，如果相关文件真的存在，应该是文部科学省派往内阁府服务的年轻职员的个人行为。对于同一事件，文部科学省和内阁府这两个日本政府的部门为何会得出完全不同的调查结果？中国国际广播电台驻日本记者朱曼君观察，大部分日本媒体都认为，此次出现两份内容相互矛盾的内部调查报告，显示出首相官邸希望将事件的责任推诿给官僚，而文部科学省的官员则已经对官邸

*句子切分问题*

In [120]:
import jieba
import re

In [142]:
punctuation = '！？｡。，＂＃＄％＆＇（）＊＋－／：；＜＝＞＠［＼］＾＿｀｛｜｝～｟｠｢｣､、〃《》「」『』【】〔〕〖〗〘〙〚〛〜〝〞〟〰〾〿–—‘\'‛“”„‟…‧﹏' + '!"#$%&\'()*+,-./:;<=>?@[\\]^_`{|}~·ʔ•'
def cut_s(content, spliter):
    sentences = re.split('[{}]'.format(spliter), content)
    s_cut = []
    for s in sentences:
        s = re.sub('[{}]'.format(punctuation+'\n'), ' ', s)
        words = jieba.cut(s)
        s_ = [w for w in words if w.strip()]
        if s_:
            s_cut.append(s_)
    return s_cut

In [143]:
content_ = cut_s(content, '，。？！：；\n\r')

In [147]:
content_

[['据', '中国', '之声', '新闻纵横', '报道'],
 ['针对',
  '近日',
  '日本',
  '媒体',
  '披露',
  '政府',
  '或',
  '为',
  '首相',
  '安倍晋三',
  '的',
  '好友',
  '办学',
  '开绿灯',
  '一事'],
 ['日本', '文部', '科学', '大臣', '松野博一', '日前', '证实'],
 ['在', '文部', '科学', '省', '内部', '找到', '已', '被', '曝光', '的', '大部分', '文件', '内容'],
 ['并', '针对', '此前', '的', '调查', '没', '能', '得出结论', '向', '公众', '致歉'],
 ['据悉'],
 ['这些',
  '内部',
  '文件',
  '内容',
  '涉及',
  '由',
  '安倍',
  '好友',
  '担任',
  '理事长',
  '的',
  '加计',
  '学园',
  '计划',
  '利用',
  '国家',
  '战略',
  '特区',
  '政策',
  '设立',
  '兽医',
  '学院',
  '一事'],
 ['文件', '中说'],
 ['加速', '推进', '设立', '兽医', '学院', '是', '来自', '官邸', '最高层', '的', '指示'],
 ['不过'],
 ['日本',
  '内阁',
  '府',
  '16',
  '号',
  '也',
  '公布',
  '了',
  '有关',
  '该',
  '事件',
  '的',
  '内部',
  '调查报告'],
 ['令人', '匪夷所思', '的', '是'],
 ['两份', '出自', '日本政府', '不同', '部门', '的', '就', '同一', '事件', '进行', '的', '调查报告'],
 ['竟然', '得出', '了', '相互', '矛盾', '的', '结果'],
 ['这', '究竟', '是', '怎么', '一', '回事'],
 ['这些', '内部', '文件', '和', '报告', '中', '暗藏', '着', '那些', '秘密'],
 ['事件',
  '要',
  '从

In [144]:
title_ = cut_s(title, '，。？！：；\n\r')

In [146]:
title_

[['安倍', '再陷', '办学', '丑闻', '日', '内阁', '府', '与', '文科', '省', '调查', '大相径庭']]

In [148]:
article_ = [[]]
for s in content_:
    article_[0] += s

In [153]:
article_

[['据',
  '中国',
  '之声',
  '新闻纵横',
  '报道',
  '针对',
  '近日',
  '日本',
  '媒体',
  '披露',
  '政府',
  '或',
  '为',
  '首相',
  '安倍晋三',
  '的',
  '好友',
  '办学',
  '开绿灯',
  '一事',
  '日本',
  '文部',
  '科学',
  '大臣',
  '松野博一',
  '日前',
  '证实',
  '在',
  '文部',
  '科学',
  '省',
  '内部',
  '找到',
  '已',
  '被',
  '曝光',
  '的',
  '大部分',
  '文件',
  '内容',
  '并',
  '针对',
  '此前',
  '的',
  '调查',
  '没',
  '能',
  '得出结论',
  '向',
  '公众',
  '致歉',
  '据悉',
  '这些',
  '内部',
  '文件',
  '内容',
  '涉及',
  '由',
  '安倍',
  '好友',
  '担任',
  '理事长',
  '的',
  '加计',
  '学园',
  '计划',
  '利用',
  '国家',
  '战略',
  '特区',
  '政策',
  '设立',
  '兽医',
  '学院',
  '一事',
  '文件',
  '中说',
  '加速',
  '推进',
  '设立',
  '兽医',
  '学院',
  '是',
  '来自',
  '官邸',
  '最高层',
  '的',
  '指示',
  '不过',
  '日本',
  '内阁',
  '府',
  '16',
  '号',
  '也',
  '公布',
  '了',
  '有关',
  '该',
  '事件',
  '的',
  '内部',
  '调查报告',
  '令人',
  '匪夷所思',
  '的',
  '是',
  '两份',
  '出自',
  '日本政府',
  '不同',
  '部门',
  '的',
  '就',
  '同一',
  '事件',
  '进行',
  '的',
  '调查报告',
  '竟然',
  '得出',
  '了',
  '相互',
  '矛盾',
  '的',
  '结果',
  '

In [158]:
title_v = sif_s1(title_[0])

In [167]:
article_v = sif_s1(article_[0])

In [170]:
content_v = np.zeros([len(content_), 250])

In [176]:
for i, _ in enumerate(content_v):
    content_v[i, :] = sif_s1(content_[i])

In [177]:
content_v.shape

(100, 250)

In [180]:
content_v_ = sif_s2(content_v)

In [187]:
content_v_[0,:].shape

(250,)

**求每个句子和全文以及标题的相关度**

*通过余弦相似度来计算相关度，并且分配权重*

In [235]:
def relative(sentences, title, article, weight=0.2):
    def cosine_v(v1, v2):
        num = np.dot(v1, v2.T)
        demon = np.linalg.norm(v1) * np.linalg.norm(v2)
        return num / demon
    
    c = []
    for i in range(sentences.shape[0]):
        s = sentences[i, :]
        c_i = weight * cosine_v(s, article.T) + (1 - weight) * cosine_v(s, title.T)
        c.append(c_i)
    return c

In [236]:
c = relative(content_v_, title_v, article_v)

In [192]:
c

[0.22095923387248365,
 0.6803825460737832,
 0.5049182259800864,
 0.5627258024469752,
 0.5146929667122135,
 0.07354793693078404,
 0.644195593206706,
 0.2919491071400345,
 0.4259246798635091,
 0.21162200200731313,
 0.68105732562805,
 0.06536703728818449,
 0.5716893948746704,
 0.33895127366475053,
 0.1654769724578204,
 0.4323426064251271,
 0.5581310895051512,
 0.6441053400769661,
 0.4812250241961828,
 0.5265994361681681,
 0.2919491071400345,
 0.4259246798635091,
 0.17321765582891213,
 0.4979294371393942,
 0.2188809474124074,
 0.5334095664236577,
 0.23365323854954545,
 0.5388052576678368,
 0.4307634357497245,
 0.30376517629105937,
 0.31641939527922236,
 0.618737310174276,
 0.5729017473916587,
 0.5511510738875927,
 0.4672843296040017,
 0.5973402541664445,
 0.4867288386331623,
 0.4066663876150359,
 0.39911120700927905,
 0.5153699251702701,
 0.4779728628310224,
 0.31641939527922236,
 0.5934060623582764,
 0.6532461443710698,
 0.41306328726299524,
 0.6525293958109546,
 0.40226166563637544,
 0.6

In [202]:
content_s = re.split('[{}]'.format('，？。！：；\n\r'), content)

In [205]:
content_s = [s for s in content_s if s if s !='”']

In [206]:
content_s

['据中国之声《新闻纵横》报道',
 '针对近日日本媒体披露政府或为首相安倍晋三的好友办学“开绿灯”一事',
 '日本文部科学大臣松野博一日前证实',
 '在文部科学省内部找到已被曝光的大部分文件内容',
 '并针对此前的调查没能得出结论向公众致歉',
 '据悉',
 '这些内部文件内容涉及由安倍好友担任理事长的加计学园计划利用“国家战略特区”政策设立兽医学院一事',
 '文件中说',
 '加速推进设立兽医学院是“来自官邸最高层的指示”',
 '不过',
 '日本内阁府16号也公布了有关该事件的内部调查报告',
 '令人匪夷所思的是',
 '两份出自日本政府不同部门的就同一事件进行的调查报告',
 '竟然得出了相互矛盾的结果',
 '这究竟是怎么一回事',
 '这些内部文件和报告中暗藏着那些秘密',
 '事件要从日本《朝日新闻》今年5月17日披露的内部文件说起',
 '当时被《朝日新闻》披露的部分被怀疑是文部科学省的内部文件中',
 '内容涉及由日本首相安倍晋三的好友担任理事长的加计学园',
 '计划利用“国家战略特区”政策设立兽医学院一事',
 '文件中说',
 '加速推进设立兽医学院是“来自官邸最高层的指示”',
 '据媒体报道',
 '日本政府已有50多年没有批准新设立兽医学院',
 '今年1月',
 '加计学园获批利用“国家战略特区”政策在爱媛县今治市新设兽医学院',
 '3月',
 '今治市议会通过相关议案',
 '决定为加计学园无偿提供建设用地',
 '并将提供最高达96亿日元的建设补助',
 '对此',
 '前文部科学省事务次官前川喜平5月25日晚召开记者会证实这批文件确实存在',
 '他认为文件中所提到的“官邸最高层”指的应是首相本人或官房长官',
 '并批评政府本应公平、公正的行政环境已经遭到破坏',
 '关于这份内部文件的真伪',
 '日本文部科学省在第一时间宣称相关文件并不存在',
 '但在已经辞职的元政务官和多位在职职员的证言下',
 '本月15日又改口承认',
 '确实有这样一份文件',
 '并且承认在批准新设兽医学科过程中',
 '受到了来自首相官邸的压力',
 '对此',
 '日本内阁府迅速展开自查',
 '16日内阁府发布调查资料',
 '称未能发现相关文件',
 '负责主导相关调查的地方创生大臣山本幸三甚至在国会暗示',
 '如果相

In [210]:
s_c = []
i = 0
for s_i, c_i in zip(content_s, c):
    s_c.append([i, s_i, c_i])
    i += 1

In [217]:
topn = sorted([n[0] for n in sorted(s_c, key=lambda x: x[1], reverse=True)[:10]])

In [218]:
for i in topn:
    print(content_s[i])

针对近日日本媒体披露政府或为首相安倍晋三的好友办学“开绿灯”一事
这些内部文件内容涉及由安倍好友担任理事长的加计学园计划利用“国家战略特区”政策设立兽医学院一事
这究竟是怎么一回事
这些内部文件和报告中暗藏着那些秘密
针对这些的疑虑无法完全消除
这遭也到了在野党和外界的质疑
这次事件真正的焦点应该在于批准加计学园新设兽医学部的决策到底合不合规
这其中到底有没有人违法牟利
越来越多的舆论开始转向对安倍政权的批判方向
（记者李思默）


*KNN平滑*

In [232]:
knn_c = []
def left_(i):
    if i <= 2:
        return 0
    else:
        return i - 2
for i in range(len(c)):
    c_i = sum(c[left_(i): i+3]) / len(c[left_(i): i+3])
    knn_c.append(c_i)

In [227]:
knn_c

[0.46875333530878444,
 0.4922464520933321,
 0.49673575501710837,
 0.46725349562876844,
 0.4600161050553531,
 0.41742228128734266,
 0.39006205677064953,
 0.32944786382966945,
 0.4509497415691226,
 0.3351840303854182,
 0.39113208793234544,
 0.3737374066925937,
 0.3645084007826952,
 0.3147654569421106,
 0.4133182673855039,
 0.427801456425963,
 0.45625620653224946,
 0.5284806992743191,
 0.5004019994173005,
 0.47396071748897217,
 0.37978318063936134,
 0.3831240632280036,
 0.3215803654768515,
 0.36987245733357615,
 0.33141816907078336,
 0.40453568943856827,
 0.3911024891606344,
 0.4080793349363647,
 0.36468130070747773,
 0.44169811503242384,
 0.44851741297718817,
 0.47259494060476187,
 0.5052987712673503,
 0.5614829430447946,
 0.5350812487365719,
 0.5018341767812474,
 0.47142620340558467,
 0.4810433225188383,
 0.45716984425175394,
 0.423107955580966,
 0.4604558905296141,
 0.5112828780019723,
 0.4908215504205173,
 0.5257328570165037,
 0.5429013110879344,
 0.5509452253210296,
 0.50452236893512

In [237]:
s_c = []
i = 0
for s_i, c_i in zip(content_s, knn_c):
    s_c.append([i, s_i, c_i])
    i += 1

In [229]:
topn = sorted([n[0] for n in sorted(s_c, key=lambda x: x[1], reverse=True)[:10]])
for i in topn:
    print(content_s[i])

针对近日日本媒体披露政府或为首相安倍晋三的好友办学“开绿灯”一事
这些内部文件内容涉及由安倍好友担任理事长的加计学园计划利用“国家战略特区”政策设立兽医学院一事
这究竟是怎么一回事
这些内部文件和报告中暗藏着那些秘密
针对这些的疑虑无法完全消除
这遭也到了在野党和外界的质疑
这次事件真正的焦点应该在于批准加计学园新设兽医学部的决策到底合不合规
这其中到底有没有人违法牟利
越来越多的舆论开始转向对安倍政权的批判方向
（记者李思默）


In [241]:
# w = 0.2
topn = sorted([n[0] for n in sorted(s_c, key=lambda x: x[1], reverse=True)[:8]])
for i in topn:
    print(content_s[i])

针对近日日本媒体披露政府或为首相安倍晋三的好友办学“开绿灯”一事
这究竟是怎么一回事
这些内部文件和报告中暗藏着那些秘密
针对这些的疑虑无法完全消除
这遭也到了在野党和外界的质疑
这次事件真正的焦点应该在于批准加计学园新设兽医学部的决策到底合不合规
这其中到底有没有人违法牟利
（记者李思默）


>为了保证句子连贯性，用逗号等句中符号分句不太合理